In [6]:
import numpy as np
import scipy
import h5py
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.python.keras.optimizers import Adam,RMSprop 

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Sequential

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Sequential,model_from_json
from PIL import Image

In [12]:
STARTDIR='C:\\data\\clothes\\'
adversarialexample_dir = os.path.join(STARTDIR,'adversarialexample\\')
model_dir = os.path.join(STARTDIR,'model_ourclothes.json')
weights_dir=os.path.join(STARTDIR,'weights_ourclothes.h5')
image_width, image_height = 150, 150
learning_rate = 0.2 ## How much to update the hacked image in each iteration
object_type_to_fake = 1  #Pant
input_shape = (image_width, image_height, 3) 
labels = ['hat', 'pant', 'shirt', 'shoes', 'skirt', 'socks']

In [13]:
json_file = open(model_dir, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(weights_dir)
#print(loaded_model.summary())

In [14]:
model_input_layer = loaded_model.layers[0].input  # First Layer
model_output_layer = loaded_model.layers[-1].output # Last Layer    

In [ ]:
files = os.listdir(adversarialexample_dir)
    
for file in files:
        fullFileName= os.path.join(adversarialexample_dir, file)
        print(fullFileName)
         
        img = image.load_img(fullFileName, target_size=(image_width, image_height))
        original_image = image.img_to_array(img)

   
        original_image /= 255.

        # Add a 4th dimension for batch size (as Keras expects)
        original_image = np.expand_dims(original_image, axis=0)
        #Create a copy of the input image to hack on
        adversarialEg_image = np.copy(original_image)
        
        # Pre-calculate the maximum change we will allow to the image
        max_change_above = original_image + 0.01
        max_change_below = original_image - 0.01

        # Define the cost function.
        # Our 'cost' will be the likelihood out image is the target class according to the pre-trained model
        cost_function = model_output_layer[0, object_type_to_fake]

        # We'll ask Keras to calculate the gradient based on the input image and the currently predicted class
        # In this case, referring to "model_input_layer" will give us back image we are hacking.
        gradient_function = K.gradients(cost_function, model_input_layer)[0]

        # Create a Keras function that we can call to calculate the current cost and gradient
        grab_cost_and_gradients_from_model = K.function([model_input_layer, K.learning_phase()], [cost_function, gradient_function])

        cost = 0.0     
        
        # In a loop, keep adjusting the adversarialEg image slightly so that it tricks the model more and more
        # until it gets to at least 90% confidence
        while cost < 0.85:
            # Check how close the image is to our target class and grab the gradients we
            # can use to push it one more step in that direction.
            # Note: It's really important to pass in '0' for the Keras learning mode here!
            # Keras layers behave differently in prediction vs. train modes!
            cost, gradients = grab_cost_and_gradients_from_model([adversarialEg_image, 0])

            # Move the adversarialEg image one step further towards fooling the model
            adversarialEg_image += gradients * learning_rate

            # Ensure that the image doesn't ever change too much to either look funny or to become an invalid image
            adversarialEg_image = np.clip(adversarialEg_image, max_change_below, max_change_above)
            adversarialEg_image = np.clip(adversarialEg_image, -1.0, 1.0)

            print("Adversarial Example Image cost is {:.3}%".format(cost * 100))

        # De-scale the image's pixels from [-1, 1] back to the [0, 255] range
        img = adversarialEg_image[0]
        img *= 255.
        # Save the adversarialEg image!
        im = Image.fromarray(img.astype(np.uint8))
        file = "advEx_"+file
        adversarialEgFileName= os.path.join(adversarialexample_dir, file)
        print(adversarialEgFileName)
        im.save(adversarialEgFileName)
        break